# Sistema Integrado de Processamento de Imagens

**Módulo Completo:** Transformações de Contraste + Análise de Filtros de Suavização

Este notebook combina:
- Conversão para escala de cinza e transformações (Logaritmo, Quadrado, Exponencial)
- Análise de filtros de suavização (Gaussiano, Mediano, Bilateral)
- Inversão de cores e exibição de canais RGB
- Histogramas e métricas de qualidade
- Download em lote

# Célula 1: Importação das Bibliotecas

In [ ]:
import cv2 as cv
import numpy as np
import time
import matplotlib.pyplot as plt
import pandas as pd
import ipywidgets as widgets
from google.colab import files
from IPython.display import display, clear_output
import io
import zipfile
import os

print("✅ Bibliotecas importadas com sucesso!")

# Célula 2: Definição de Todas as Funções

In [ ]:
# ========== FUNÇÕES DE CONVERSÃO E TRANSFORMAÇÃO ==========

def converter_para_cinza(img_bgr):
    """Converte uma imagem BGR para escala de cinza."""
    return cv.cvtColor(img_bgr, cv.COLOR_BGR2GRAY)

def aplicar_transformacao(img_cinza, metodo):
    """Aplica transformações de contraste na imagem em escala de cinza."""
    img_float = img_cinza.astype(np.float64)

    if metodo == 'Logaritmo':
        denominador = np.log(1 + np.max(img_float))
        if denominador == 0:
            return img_cinza
        c = 255 / denominador
        img_transformada = c * np.log(1 + img_float)
    elif metodo == 'Quadrado':
        img_norm = img_float / 255.0
        img_transformada = np.power(img_norm, 2.0) * 255
    elif metodo == 'Exponencial':
        img_norm = img_float / 255.0
        img_transformada = np.power(img_norm, 0.5) * 255
    else:
        return img_cinza
    return np.uint8(img_transformada)

def inverter_cores(img_cinza):
    """Inverte as cores da imagem (efeito negativo)."""
    return cv.bitwise_not(img_cinza)

def exibir_canal_cor(img_bgr, canal):
    """Exibe um canal de cor específico (R, G, B)."""
    azul, verde, vermelho = cv.split(img_bgr)
    if canal == 'Vermelho':
        return vermelho
    elif canal == 'Verde':
        return verde
    elif canal == 'Azul':
        return azul
    else:
        return converter_para_cinza(img_bgr)

def plotar_histograma(img_cinza):
    """Calcula e plota o histograma de uma imagem em escala de cinza."""
    plt.figure(figsize=(6, 4))
    plt.title("Histograma de Tons de Cinza")
    plt.xlabel("Intensidade")
    plt.ylabel("Número de Pixels")
    plt.hist(img_cinza.ravel(), bins=256, range=[0, 256])
    plt.xlim([0, 256])
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.show()

# ========== FUNÇÕES DE FILTROS E ANÁLISE ==========

def medir_ruido(img):
    """Calcula o nível de ruído como o desvio padrão dos pixels."""
    return np.std(img)

def medir_foco_laplaciano(img_cinza):
    """Avalia a nitidez pela variância do Laplaciano.
    Valores altos indicam maior nitidez."""
    return cv.Laplacian(img_cinza, cv.CV_64F).var()

def aplicar_filtros(img_cinza):
    """Aplica os filtros: Gaussiano, Mediano e Bilateral."""
    img_gaussiano = cv.GaussianBlur(img_cinza, (5, 5), 0)
    img_mediano = cv.medianBlur(img_cinza, 5)
    img_bilateral = cv.bilateralFilter(img_cinza, 9, 75, 75)

    filtros = {
        'Gaussiano': img_gaussiano,
        'Mediano': img_mediano,
        'Bilateral': img_bilateral
    }
    return filtros

def avaliar_qualidade(df_resultados):
    """Avalia e compara a qualidade dos filtros aplicados."""
    ruido_orig = df_resultados.loc[df_resultados['Análise'] == 'Original', 'Ruído'].values[0]
    foco_orig = df_resultados.loc[df_resultados['Análise'] == 'Original', 'Foco'].values[0]

    df_filtros = df_resultados[df_resultados['Análise'] != 'Original'].copy()

    df_filtros['Redução Ruído (%)'] = ((ruido_orig - df_filtros['Ruído']) / ruido_orig) * 100
    df_filtros['Perda Foco (%)'] = ((foco_orig - df_filtros['Foco']) / foco_orig) * 100
    df_filtros['Pontuação (Score)'] = df_filtros['Redução Ruído (%)'] - df_filtros['Perda Foco (%)']

    melhor_filtro = df_filtros.loc[df_filtros['Pontuação (Score)'].idxmax()]

    relatorio = f"O filtro {melhor_filtro['Análise']} foi o mais eficaz, com {melhor_filtro['Redução Ruído (%)']:.2f}% de redução de ruído e {melhor_filtro['Perda Foco (%)']:.2f}% de perda de foco."

    df_final = pd.concat([df_resultados.loc[df_resultados['Análise'] == 'Original'], df_filtros])
    df_final = df_final.set_index('Análise')
    df_final = df_final.fillna('-')

    return relatorio, df_final

# ========== VARIÁVEIS GLOBAIS ==========

imagens_carregadas = {}
imagem_selecionada_bgr = None
nome_arquivo_selecionado = ""
imagem_processada_final = None
download_lock = False

print("✅ Todas as funções definidas com sucesso!")

# Célula 3: Upload de Múltiplas Imagens

In [ ]:
imagens_carregadas = {}
imagem_selecionada_bgr = None
nome_arquivo_selecionado = ""

print("Por favor, selecione uma ou mais imagens (JPG, JPEG, PNG).")
uploaded = files.upload()

if uploaded:
    arquivos_validos = 0
    for nome_arquivo, dados_arquivo in uploaded.items():
        extensao = nome_arquivo.split('.')[-1].lower()

        if extensao in ['jpg', 'jpeg', 'png']:
            try:
                buffer_imagem = np.frombuffer(dados_arquivo, np.uint8)
                imagem_bgr = cv.imdecode(buffer_imagem, cv.IMREAD_COLOR)

                imagens_carregadas[nome_arquivo] = imagem_bgr
                arquivos_validos += 1

            except Exception as e:
                print(f"\n❌ Erro ao processar '{nome_arquivo}': Arquivo corrompido ou inválido.")
        else:
            print(f"\n❌ Erro: Arquivo '{nome_arquivo}' IGNORADO. Formato não suportado.")

    if arquivos_validos > 0:
        print(f"\n✅ {arquivos_validos} imagem(ns) carregada(s) com sucesso!")
        print("➡️ Execute a próxima célula para VISUALIZAR e SELECIONAR a imagem ativa.")
    else:
        print("\n❌ Nenhuma imagem válida foi carregada.")
else:
    print("\n❌ Nenhuma imagem foi selecionada.")

# Célula 4: Visualização Inicial de Todas as Imagens

In [ ]:
if imagens_carregadas:
    print(f"Exibindo preview das {len(imagens_carregadas)} imagens carregadas...")
    print("-" * 40)

    for nome_arquivo, imagem_bgr in imagens_carregadas.items():
        imagem_rgb = cv.cvtColor(imagem_bgr, cv.COLOR_BGR2RGB)
        imagem_cinza = converter_para_cinza(imagem_bgr)

        fig, axes = plt.subplots(1, 2, figsize=(12, 6))

        axes[0].imshow(imagem_rgb)
        axes[0].set_title(f'Original: {nome_arquivo}')
        axes[0].axis('off')

        axes[1].imshow(imagem_cinza, cmap='gray')
        axes[1].set_title('Convertida (Escala de Cinza)')
        axes[1].axis('off')

        plt.tight_layout()
        plt.show()

        altura, largura, _ = imagem_bgr.shape
        print(f"Dimensões: {largura}px (largura) x {altura}px (altura)")
        print("-" * 40)

else:
    print("❌ Nenhuma imagem carregada. Execute a Célula 3 para fazer o upload primeiro.")

# Célula 5: Seleção da Imagem Ativa

In [ ]:
if imagens_carregadas:
    seletor_imagem = widgets.Dropdown(
        options=list(imagens_carregadas.keys()),
        description='Imagem Ativa:',
        disabled=False,
    )

    output_selecao = widgets.Output()

    def ao_selecionar_imagem(change):
        global imagem_selecionada_bgr, nome_arquivo_selecionado

        nome_arquivo_selecionado = change['new']
        imagem_selecionada_bgr = imagens_carregadas[nome_arquivo_selecionado]

        with output_selecao:
            clear_output(wait=True)
            imagem_rgb = cv.cvtColor(imagem_selecionada_bgr, cv.COLOR_BGR2RGB)

            plt.figure(figsize=(7, 5))
            plt.imshow(imagem_rgb)
            plt.title(f"Visualizando: {nome_arquivo_selecionado}")
            plt.axis('off')
            plt.show()

            print("\n✅ Imagem selecionada! Prossiga para as próximas células para processá-la.")

    seletor_imagem.observe(ao_selecionar_imagem, names='value')

    print("Selecione a imagem que deseja processar:")
    display(seletor_imagem, output_selecao)
    ao_selecionar_imagem({'new': seletor_imagem.value})

else:
    print("❌ Nenhuma imagem carregada. Execute a Célula 3 para fazer o upload primeiro.")

# Célula 6: MÓDULO 1 - Transformações de Contraste e Manipulações

In [ ]:
if imagem_selecionada_bgr is not None:

    algoritmo_dropdown = widgets.Dropdown(
        options=['Nenhum', 'Logaritmo', 'Quadrado', 'Exponencial'],
        value='Nenhum',
        description='Transformação:',
        disabled=False,
    )

    inverter_checkbox = widgets.Checkbox(
        value=False,
        description='Inverter Cores (Negativo)',
        disabled=False
    )

    output_area = widgets.Output()

    def on_controls_change(change):
        global imagem_processada_final

        with output_area:
            clear_output(wait=True)

            img_cinza = converter_para_cinza(imagem_selecionada_bgr)

            metodo = algoritmo_dropdown.value
            img_transformada = aplicar_transformacao(img_cinza, metodo)

            if inverter_checkbox.value:
                img_transformada = inverter_cores(img_transformada)

            imagem_processada_final = img_transformada

            imagem_original_rgb = cv.cvtColor(imagem_selecionada_bgr, cv.COLOR_BGR2RGB)

            fig, axes = plt.subplots(1, 2, figsize=(14, 7))

            axes[0].imshow(img_transformada, cmap='gray')
            axes[0].set_title(f"Imagem Processada\n(Transformação: {metodo})")
            axes[0].axis('off')

            axes[1].imshow(imagem_original_rgb)
            axes[1].set_title("Imagem Original (RGB)")
            axes[1].axis('off')

            plt.tight_layout()
            plt.show()

            plotar_histograma(img_transformada)

    algoritmo_dropdown.observe(on_controls_change, names='value')
    inverter_checkbox.observe(on_controls_change, names='value')

    print("=" * 60)
    print("MÓDULO 1: Transformações de Contraste")
    print("=" * 60)
    print("Use os controles abaixo para manipular a imagem.\n")
    display(widgets.VBox([
        widgets.HBox([algoritmo_dropdown, inverter_checkbox]),
        output_area
    ]))

    on_controls_change(None)

else:
    print("❌ Nenhuma imagem selecionada. Execute a Célula 3 (Upload) e a Célula 5 (Seleção) primeiro.")

# Célula 7: MÓDULO 2 - Análise de Filtros de Suavização

In [ ]:
if imagem_selecionada_bgr is not None:
    print("=" * 60)
    print("MÓDULO 2: Análise de Filtros de Suavização")
    print("=" * 60)
    print(f"Analisando: {nome_arquivo_selecionado}")
    print("-" * 60)

    img_cinza_original = converter_para_cinza(imagem_selecionada_bgr)
    ruido_original = medir_ruido(img_cinza_original)
    foco_original = medir_foco_laplaciano(img_cinza_original)

    resultados = {
        'Análise': ['Original'],
        'Ruído': [ruido_original],
        'Foco': [foco_original]
    }

    imagens_filtradas = aplicar_filtros(img_cinza_original)
    print("✓ Filtros aplicados...")

    for nome_filtro, img_filtrada in imagens_filtradas.items():
        ruido_filtro = medir_ruido(img_filtrada)
        foco_filtro = medir_foco_laplaciano(img_filtrada)

        resultados['Análise'].append(nome_filtro)
        resultados['Ruído'].append(ruido_filtro)
        resultados['Foco'].append(foco_filtro)

    df_resultados = pd.DataFrame(resultados)
    relatorio_final, df_tabela = avaliar_qualidade(df_resultados)

    print("✓ Análise concluída.")
    print("-" * 60)

    # Visualização comparativa
    fig, axes = plt.subplots(2, 2, figsize=(14, 12))
    fig.suptitle(f"Comparação de Filtros: {nome_arquivo_selecionado}", fontsize=16)

    axes[0, 0].imshow(img_cinza_original, cmap='gray')
    axes[0, 0].set_title(f"Original (Cinza)\nRuído: {ruido_original:.2f} | Foco: {foco_original:.2f}")
    axes[0, 0].axis('off')

    dados_gauss = df_tabela.loc['Gaussiano']
    axes[0, 1].imshow(imagens_filtradas['Gaussiano'], cmap='gray')
    axes[0, 1].set_title(f"Filtro Gaussiano\nRuído: {dados_gauss['Ruído']:.2f} | Foco: {dados_gauss['Foco']:.2f}")
    axes[0, 1].axis('off')

    dados_median = df_tabela.loc['Mediano']
    axes[1, 0].imshow(imagens_filtradas['Mediano'], cmap='gray')
    axes[1, 0].set_title(f"Filtro Mediano\nRuído: {dados_median['Ruído']:.2f} | Foco: {dados_median['Foco']:.2f}")
    axes[1, 0].axis('off')

    dados_bilat = df_tabela.loc['Bilateral']
    axes[1, 1].imshow(imagens_filtradas['Bilateral'], cmap='gray')
    axes[1, 1].set_title(f"Filtro Bilateral\nRuído: {dados_bilat['Ruído']:.2f} | Foco: {dados_bilat['Foco']:.2f}")
    axes[1, 1].axis('off')

    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    plt.show()

    # Tabela comparativa
    print("\n" + "="*60)
    print("Tabela Comparativa de Desempenho")
    print("="*60)

    format_percent = lambda v: f'{v:.2f}%' if isinstance(v, (int, float)) else v
    format_score = lambda v: f'{v:.2f}' if isinstance(v, (int, float)) else v

    display(df_tabela.style
        .map(lambda v: 'color: green' if isinstance(v, (int, float)) and v > 0 else ('color: red' if isinstance(v, (int, float)) and v < 0 else None),
             subset=['Redução Ruído (%)', 'Perda Foco (%)', 'Pontuação (Score)'])
        .format({
            'Ruído': '{:.2f}',
            'Foco': '{:.2f}',
            'Redução Ruído (%)': format_percent,
            'Perda Foco (%)': format_percent,
            'Pontuação (Score)': format_score
        })
    )

    print("\n" + "="*60)
    print("🏆 Resultado da Avaliação")
    print("="*60)
    print(f"\n{relatorio_final}\n")

else:
    print("❌ Nenhuma imagem selecionada. Execute a Célula 3 (Upload) e a Célula 5 (Seleção) primeiro.")

# Célula 8: Download em Lote (ZIP)

In [ ]:
clear_output(wait=True)

DOWNLOAD_EM_ANDAMENTO = False

if 'imagens_carregadas' in globals() and imagens_carregadas:

    def iniciar_download_em_lote_zip(b):
        global DOWNLOAD_EM_ANDAMENTO

        if DOWNLOAD_EM_ANDAMENTO:
            with output_download:
                 print("⚠️ Processo já em andamento! Aguarde a finalização.")
            return

        DOWNLOAD_EM_ANDAMENTO = True
        botao_download.disabled = True

        try:
            with output_download:
                clear_output(wait=True)
                print("🚀 Iniciando processo de compactação...")

                escolha = escolha_download.value
                metodo_atual = 'Nenhum'
                inverter_atual = False

                if 'algoritmo_dropdown' in globals():
                    metodo_atual = algoritmo_dropdown.value
                if 'inverter_checkbox' in globals():
                    inverter_atual = inverter_checkbox.value

                nome_arquivo_zip = f"imagens_processadas_{int(time.time())}.zip"

                with zipfile.ZipFile(nome_arquivo_zip, 'w') as zf:
                    total = len(imagens_carregadas)
                    for i, (nome_arquivo, imagem_bgr) in enumerate(imagens_carregadas.items()):
                        print(f"[{i+1}/{total}] Processando: {nome_arquivo}...")
                        imagem_final = None
                        sufixo_nome = ""

                        if escolha == 'Aplicar Transformação Atual (em todas)':
                            sufixo_nome = f"_{metodo_atual.lower()}"
                            if inverter_atual:
                                sufixo_nome += "_invertido"
                            img_cinza = converter_para_cinza(imagem_bgr)
                            img_transformada = aplicar_transformacao(img_cinza, metodo_atual)
                            if inverter_atual:
                                img_transformada = inverter_cores(img_transformada)
                            imagem_final = img_transformada
                        else:
                            sufixo_nome = "_cinza"
                            imagem_final = converter_para_cinza(imagem_bgr)

                        if imagem_final is not None:
                            nome_base, extensao = os.path.splitext(nome_arquivo)
                            formato_saida = '.png' if extensao.lower() == '.png' else '.jpg'
                            nome_arquivo_saida = f"{nome_base}{sufixo_nome}{formato_saida}"

                            is_success, buffer_saida = cv.imencode(formato_saida, imagem_final)
                            if is_success:
                                zf.writestr(nome_arquivo_saida, buffer_saida.tobytes())
                                print(f"✔️ Adicionado '{nome_arquivo_saida}' ao ZIP.")
                            else:
                                print(f"❌ Falha ao codificar: {nome_arquivo}")

                print("\n" + "="*40)
                print(f"✅ Arquivo '{nome_arquivo_zip}' criado! Iniciando download...")
                files.download(nome_arquivo_zip)

        finally:
            print("\n✓ Processo concluído!")
            DOWNLOAD_EM_ANDAMENTO = False
            botao_download.disabled = False

    label_download = widgets.Label(value="Opção de Download em Lote:")
    escolha_download = widgets.RadioButtons(
        options=['Aplicar Transformação Atual (em todas)', 'Converter para Cinza Padrão (em todas)'],
        description='',
        disabled=False
    )
    botao_download = widgets.Button(
        description="📦 Gerar e Baixar .ZIP",
        button_style='success',
        tooltip='Processar e baixar um arquivo ZIP com todas as imagens',
        icon='download'
    )
    output_download = widgets.Output()

    botao_download.on_click(iniciar_download_em_lote_zip)

    print("=" * 60)
    print("MÓDULO DE DOWNLOAD EM LOTE")
    print("=" * 60)
    print("Selecione como deseja processar TODAS as imagens carregadas:\n")
    box_opcoes = widgets.VBox([label_download, escolha_download])
    display(box_opcoes, botao_download, output_download)

else:
    print("❌ Nenhuma imagem carregada. Execute a Célula 3 (Upload) primeiro.")